In [ ]:
!pip install gensim

In [ ]:
!pip install --upgrade pip setuptools wheel

In [ ]:
!python --version

Python 3.11.11


In [ ]:
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd
import gensim

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Gensim version:", gensim.__version__)

NumPy version: 1.26.4
Pandas version: 2.2.2
Gensim version: 4.3.3


In [ ]:
#pip uninstall pandas
#pip uninstall gensim
#pip uninstall numpy
# Install compatible versions
#!pip install numpy==1.23.0 pandas==1.5.0 gensim==4.0.0

Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pandas/*
Proceed (Y/n)? Y
  Successfully uninstalled pandas-2.2.2


In [ ]:
!pip install python-docx nltk

In [ ]:
# Download NLTK resources
import nltk # Import the nltk library
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

**1. Word Embeddings (Word2Vec/GloVe/top2vec):**

**2. Search Functionality (Custom Search with Embeddings):**

In [ ]:
# Load the Word documents
import os
import docx
import re

# Path to the folder containing Word documents
docx_folder = "/content/sample_data/word files"

def extract_text_from_docx(file_path):
    """Extracts text from a Word (.docx) file."""
    doc = docx.Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

corpus = []
for file in os.listdir(docx_folder):
    if file.endswith(".docx"):
        text = extract_text_from_docx(os.path.join(docx_folder, file))
        corpus.append(text)

def clean_text(text):
    """Removes extra spaces, new lines, and special characters."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = text.strip()
    return text

corpus = [clean_text(doc) for doc in corpus]

#Preprocess the extracted text

# Preprocessing function
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Preprocess the corpus
processed_corpus = [preprocess(doc) for doc in corpus]  # List of tokenized sentence
#print(processed_corpus)

# Train a Word2Vec model
word2vec_model = Word2Vec(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Function to convert a document or query into a vector
def text_to_vector(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Function to rank documents based on similarity to the query
def search(query, model, corpus, top_k=5):
    # Preprocess the query
    processed_query = preprocess(query)
    # Convert query to vector
    query_vector = text_to_vector(processed_query, model)
    # Convert documents to vectors
    doc_vectors = [text_to_vector(doc, model) for doc in processed_corpus]
    # Compute cosine similarity between query and documents
    similarities = [cosine_similarity([query_vector], [doc_vec])[0][0] for doc_vec in doc_vectors]
    # Rank documents by similarity
    ranked_indices = np.argsort(similarities)[::-1]
    # Ensure top_k does not exceed the number of documents
    top_k = min(top_k, len(corpus))
    # Return top-k results
    results = [(corpus[i], similarities[i]) for i in ranked_indices[:top_k]]
    return results

# Example query
query = input("Enter your search query: ")
results = search(query, word2vec_model, corpus, top_k=3)

# Display results
for i, (doc, score) in enumerate(results):
    print(f"Result {i+1} (Score: {score:.4f}):")
    print(doc)
    print()

Enter your search query: define photosynthesis
Result 1 (Score: 0.9098):
Photosynthesis Photosynthesis is the process by which plants, some bacteria and some protistans use the energy from sunlight to produce glucose from carbon dioxide and water. This glucose can be converted into pyruvate which releases adenosine triphosphate (ATP) by cellular respiration. Oxygen is also formed. Photosynthesis may be summarised by the word equation: The conversion of usable sunlight energy into chemical energy is associated with the action of the green pigment chlorophyll. Chlorophyll is a complex molecule. Several modifications of chlorophyll occur among plants and other photosynthetic organisms. All photosynthetic organisms have chlorophyll a. Accessory pigments absorb energy that chlorophyll a does not absorb. Accessory pigments include chlorophyll b (also c, d, and e in algae and protistans), xanthophylls, and carotenoids (such as beta-carotene). Chlorophyll a absorbs its energy from the violet-b

**3. Fine-Tuning with Transformers (Using Huggingface Transformers or Unsloth):**

In [ ]:
# Step 1: Install required libraries
!pip install python-docx transformers datasets torch

# Step 2: Extract text from uploaded Word documents
from docx import Document
import os

docx_folder = "/content/sample_data/word files"

uploaded_files = [f for f in os.listdir(docx_folder) if f.endswith(".docx")]

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

docx_texts = {}
for docx_name in uploaded_files:

    docx_path = os.path.join(docx_folder, docx_name)
    docx_texts[docx_name] = extract_text_from_docx(docx_path)

def preprocess(text):
    """Removes extra spaces, new lines, special characters,
       tokenizes text, and removes stop words."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = text.strip()
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    processed_text = " ".join(tokens)
    return processed_text

processed_corpus = {}
for docx_name, text in docx_texts.items():
    processed_corpus[docx_name] = preprocess(text)

# Step 4: Prepare the dataset
corpus_text = " ".join([" ".join(tokens) for tokens in processed_corpus.values()])

with open("corpus.txt", "w") as f:
    f.write(corpus_text)

# Step 5: Load a pre-trained GPT-2 model and tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Add the padding token to the tokenizer
special_tokens_dict = {'pad_token': '<pad>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

model = GPT2LMHeadModel.from_pretrained(model_name)
# resize the model's embedding layer to match the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

# Step 6: Tokenize the dataset
from datasets import load_dataset

dataset = load_dataset("text", data_files="corpus.txt")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Step 7: Set up training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
 #   prediction_loss_only=True,
)

# Step 8: Define a trainer
from transformers import Trainer
import torch

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Computes the loss for language modeling.
        """
        outputs = model(**inputs)  # Get model outputs
        # Ensure loss is not detached
        loss = outputs.loss
        if loss is not None:
            loss = loss.requires_grad_(True) # Ensure the loss requires gradients
        # Check if loss is None and return a zero tensor if it is
        if loss is None:
            loss = torch.tensor(0.0, device=model.device, requires_grad=True) # Return a zero tensor with requires_grad=True instead of None
        return (loss, outputs) if return_outputs else loss


trainer = CustomTrainer( # Use the custom Trainer class
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
)

# Step 9: Fine-tune the model
trainer.train()

# Step 10: Save the fine-tuned model
model.save_pretrained("./fine-tuned-gpt2")
tokenizer.save_pretrained("./fine-tuned-gpt2")

# Step 11: Load and use the fine-tuned model
from transformers import pipeline

fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine-tuned-gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine-tuned-gpt2")

generator = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

query = input("Enter your query: ")

response = generator(query, max_length=50, num_return_sequences=1)
print("Generated Response:")
print(response[0]["generated_text"])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-38-57678f7948ae>:108: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer( # Use the custom Trainer class


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


Device set to use cpu


Enter your query: deep learning


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Response:
deep learning with deep understanding of complex systems, but the underlying neural basis can be refined at the individual level with deep learning with machine learning. Deep learning is becoming more widespread as a method for building and managing complex networks in deep learning environments.


